# Video Watermarking: Examples

This notebook shows how we can use Omniseal Bench Python APIs to watermark video with different models, and to apply attacks then detect the watermarks and print out different metrics

## Example 1: Quick start

In the following example, we build a custom dataset, then use public Videoseal model to watermark each video. Next, we run the attacks and detect if watermarks surive them. Finally, we report the quality metrics and detection scores.

Make sure to first install the public videoseal package: `pip install git+https://github.com/facebookresearch/videoseal.git`

In [1]:
# Some Jupyter kernels check TORCH_DISTRIBUTED_DEBUG and throw an error if not found
import os
os.environ["TORCH_DISTRIBUTED_DEBUG"] = "OFF"

In [2]:
import sys
sys.executable

'/private/home/tuantran/.conda/envs/omnisealbench/bin/python'

In [3]:
import shutil
from pathlib import Path

# This notebook runs inside the root directory of omnisealbench. Adjust this when you run in a different location

def build_test_video_dataset():
    # Make up a fake local dataset from test image
    example_video = "../examples/videos/vid_1.mp4"

    tmp_path = Path("tmp/test_videos")
    tmp_path.mkdir(exist_ok=True, parents=True)

    shutil.copy(example_video, tmp_path / "vid_1_1.mp4")
    shutil.copy(example_video, tmp_path / "vid_1_2.mp4")
    shutil.copy(example_video, tmp_path / "vid_1_3.mp4")
    shutil.copy(example_video, tmp_path / "vid_1_4.mp4")
    shutil.copy(example_video, tmp_path / "other_random_vid_1.mp4")
    shutil.copy(example_video, tmp_path / "other_random_vid_2.mp4")
    shutil.copy(example_video, tmp_path / "other_random_vid_3.mp4")
    shutil.copy(example_video, tmp_path / "other_random_vid_4.mp4")
    
build_test_video_dataset()

In [4]:
from omnisealbench import AttackConfig, task, get_model

attacks = [
    AttackConfig(name="JPEG", data_type="video", args={"quality": 40}),
    AttackConfig(name="GaussianBlur", data_type="video", args={"kernel_size": 9}),
]

e2e = task(
    "default",
    modality="video",
    
    # Dataset options
    dataset_dir="tmp/test_videos",
    
    # data loading options
    batch_size=1,
    num_workers=0,

    overwrite=True,
    
    output_resolution=256,
    max_frames=5,
    fps=24,
    crf=0,
    
    # watermark options:
    # message_size (Optional): THis is only specified if the model does not have the attribute 'nbits'
    how_to_generate_message="per_dataset",  # Options: 'per_dataset', 'per_batch'
    
    # detection options
    detection_bits=16,  # how many leading bits are reserved to calculate the detection accuracy etc.
    attacks=attacks,

    seed=42,

    # Put `metrics="all"` for computing all metrics ("psnr", "ssim", "msssim", "lpips", "vmaf")
    metrics=["psnr", "ssim"],
    
    metrics_device="cpu",
    quality_metrics_chunk_size=1,  # Use chunking for large videos
)

model = get_model(
    'videoseal_0.0',    
    attenuation="jnd_1_1",
    attenuation_config="../src/omnisealbench/models/videoseal_src/configs/attenuation.yaml",
    device="cuda"
)

avg_results, raw_results = e2e(model)

omnisealbench.models.videoseal.build_model has the following dependencies: ['timm', 'pycocotools', 'einops', 'opencv-python', 'safetensors', 'av']


/private/home/tuantran/.conda/envs/omnisealbench/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running VideoWatermarkAttacksAndDetection with attack: identity
Running VideoWatermarkAttacksAndDetection with attack: JPEG__quality_40
Running VideoWatermarkAttacksAndDetection with attack: GaussianBlur__kernel_size_9


### Explanation:

In the above example, we load a local video directory, then create a task object `e2e` and model object `model`, follow by loading the model and running `__call__()` function of `e2e` to perform the end-to-end evaluation. The result is a tuple of 2 items:

1) Average metrics: For each metric, we compute the average score over the evaluation data, and return:
    - `avg`: mean value
    - `count`: Number of items
    - `square`: Mean of the squares of the values
    - `avg_ci_fn`: A function applied to the bounds of the confidence interval

2) Raw scores: For each attack, we have a dictionary of a metric scores, while each score, we have the full list of `n` individual scores.

In [5]:
avg_results

{'watermark_det_score': AverageMetric(avg=0.5833333333333334, count=24, square=0.3776041666666667, avg_ci_fn=None),
 'watermark_det': AverageMetric(avg=0.6666666666666666, count=24, square=0.6666666666666666, avg_ci_fn=None),
 'fake_det_score': AverageMetric(avg=0.3958333333333333, count=24, square=0.15755208333333334, avg_ci_fn=None),
 'fake_det': AverageMetric(avg=0.0, count=24, square=0.0, avg_ci_fn=None),
 'bit_acc': AverageMetric(avg=0.7208333412806193, count=24, square=0.5322395958503089, avg_ci_fn=None),
 'word_acc': AverageMetric(avg=0.0, count=24, square=0.0, avg_ci_fn=None),
 'p_value': AverageMetric(avg=0.052384470098776, count=24, square=0.008232385512151415, avg_ci_fn=None),
 'capacity': AverageMetric(avg=15.169441223144531, count=24, square=334.5317170873556, avg_ci_fn=None),
 'log10_p_value': AverageMetric(avg=-5.300259666961741, count=24, square=38.44463424166045, avg_ci_fn=None),
 'psnr': AverageMetric(avg=45.62201182047526, count=24, square=2095.5264193275266, avg_ci_

Above we have 8 data items, 3 attacks (the default "no-attack" is added internally and will always be run), so 24 items in total. `avg_results` has 15 metrics, falling in:

- Quality metrics: Defined by `metrics` ("ssim", "psnr")

- Profiling time: "qual_ime" (average time to run quality metrics), "det_time" (average time to run detection), "attack_time" (average time to run attacks), "decoder_time" (average time to decode messages)

- Detection scores: (All remaining scores)

In [6]:
len(raw_results)

3

In [7]:
raw_results[0]

defaultdict(list,
            {'watermark_det_score': [0.75,
              0.75,
              0.75,
              0.75,
              0.75,
              0.75,
              0.75,
              0.75],
             'watermark_det': [True, True, True, True, True, True, True, True],
             'fake_det_score': [0.375,
              0.375,
              0.375,
              0.375,
              0.375,
              0.375,
              0.375,
              0.375],
             'fake_det': [False,
              False,
              False,
              False,
              False,
              False,
              False,
              False],
             'bit_acc': [0.8125,
              0.8125,
              0.8125,
              0.8125,
              0.8125,
              0.8125,
              0.8125,
              0.8125],
             'word_acc': [False,
              False,
              False,
              False,
              False,
              False,
              False,
   

Here `raw_results` has 3 dictionary for each attack ("no-attack" and 2 defined attacks). For each dictionary, we have 15 keys corresponding to 15 metrics, and each value is a list of 8 scores for 8 items

<hr/>

### Concepts:
#### Task:

Omniseal Bench Python API consists of 2 main building blocks: **Task** and **Model**. The Task defines the dataset to be used for the evaluation, how to load data (batch size, padding, etc.) and metrics to run. 

To construct the task, we use `omnisealbench.task()`. To construct the task, you must specify at least 2 arguments: Task type and modality. The modality can be "audio", "image", "video". The task type can be one of the three:

- **generation**: Run a generator over specific dataset to watermark each of its items, and return the quality metrics of the watermarked audios.

- **detection**: Apply a specific set of attacks over a watermarked dataset, then run detector and report the detection scores (robustness) and as well as quality metrics for each attack.

- **default**: Run end-to-end genration and detection.

You can also look all available tasks with `omnisealbench.list_tasks` API:

In [8]:
from omnisealbench import list_tasks

list_tasks()

- "('generation', 'audio')": Running audio watermarking on a HuggingFace dataset and evaluating the quality of the generated watermarks. (class/function: omnisealbench.tasks.audio.evaluate_audio_watermark_generation)
- "('detection', 'audio')": Running audio watermarking on a custom dataset and evaluating the quality and robustness of the generated watermarks. (class/function: omnisealbench.tasks.audio.evaluate_audio_watermark_attacks_and_detection)
- "('default', 'audio')": Running audio watermarking end-to-end (class/function: omnisealbench.tasks.audio.evaluate_audio_watermark_end2end)
- "('default', 'image')": Running image watermarking end-to-end (class/function: omnisealbench.tasks.image.evaluate_image_watermark_end2end)
- "('generation', 'image')": Running image watermarking on a local dataset and evaluating the quality of the generated watermarks. (class/function: omnisealbench.tasks.image.evaluate_image_watermark_generation)
- "('detection', 'image')": Running image watermarkin

Each task expects a specific list of arguments, falling into three categories:
- _input data_: Specify how dataset is loaded into the task. Note that the dataset is only defined at the task definition task, the actual data loading happens when we execute the task.

- _execution_: Define the behaviour of the execution such as metrics, attacks, device, seed for reproducibility, 

- _output handling_: Define how the results are handled. These includes result directory and filename, whether to keep intermediate data in cache, whether we should log details some small subset of data for inspection, etc.


To get list of arguments for a task, call "explain()":

In [9]:
from omnisealbench import explain

print(explain("default", modality="video"))

#### Model

A model is a wrapper over the watermarking model, that converts the watermarking output to the format that the task understands. Please refer to the notebook [Model.ipynb](Model.ipynb) for more detailed instructions.

Omniseal Bench provides some ready-to-use wrapper for the popular Audio watermarking models: 
- [RivaGAN](https://github.com/DAI-Lab/RivaGAN)
- [VideoSeal](https://github.com/facebookresearch/videoseal)

The models are registered via model cards in "src/omnisealbench/cards". To get the model, we call `get_model()` and pass the name of the model card (e.g. "rivagan", "videoseal_1.0"), plus any free argument to override the attributes defined in the card.


#### Attacks:

For each modality, we have implemented different attacks in `omnisealbench.attacks`. The attack can be further parameterized to different _attack variant_, with concrete function parameter values. 

For convenience, we provide some fixed set of attacks and attack variants in the _attack registry_, which is a YAML file defining how to create the attack and attack variants. These registries can be found in "src/omnisealbench/attacks" directory (For example, "src/omnisealbench/attacks/video_attacks.yaml").


For more details on attacks, see the notebook [Attacks](Attacks.ipynb).

## Example 2: Caching and saving data

By default the API will not save the final result and data. To save the data to external files for further inspection, we pass the "cache_dir" and "result_dir":

In [10]:
# We run the notebook within the root directory, adjust the directory path
from omnisealbench import task, get_model
import warnings

# Suppress all warnings in this cell
warnings.filterwarnings("ignore")

e2e = task(
    "default",
    modality="video",
    
    # Dataset options
    dataset_dir="../examples/videos",
    
    # data loading options
    batch_size=2,
    num_workers=2,
    
    output_resolution=256,
    max_frames=5,
    fps=24,
    crf=0,    
    
    # watermark options:
    # message_size (Optional): THis is only specified if the model does not have the attribute 'nbits'
    how_to_generate_message="per_dataset",  # Options: 'per_dataset', 'per_batch'
    
    # detection options
    detection_bits=16,  # how many leading bits are reserved to calculate the detection accuracy etc.
    attacks="../tests/video/video_attacks_mini.yaml",

    cache_dir="watermark_video_results",
    result_dir="final_video_results",
    overwrite=True,
    ids_to_save="all",  # Save all videos

    seed=42,
    metrics=["psnr", "ssim"],
)

model = get_model(
    'videoseal_0.0',    
    attenuation="jnd_1_1",
    attenuation_config="../src/omnisealbench/models/videoseal_src/configs/attenuation.yaml",
    device="cuda"
)

avg_results, raw_results = e2e(model)


omnisealbench.models.videoseal.build_model has the following dependencies: ['timm', 'pycocotools', 'einops', 'opencv-python', 'safetensors', 'av']
Result watermark_video_results exists and will be overriden as --overwrite is set
Result final_video_results exists and will be overriden as --overwrite is set
Running VideoWatermarkAttacksAndDetection with attack: identity
Running VideoWatermarkAttacksAndDetection with attack: HorizontalFlip
Running VideoWatermarkAttacksAndDetection with attack: Resize__size_0.55
Running VideoWatermarkAttacksAndDetection with attack: Perspective__distortion_scale_0.5
Running VideoWatermarkAttacksAndDetection with attack: Brightness__factor_1.5
Running VideoWatermarkAttacksAndDetection with attack: Hue__factor_0.25
Running VideoWatermarkAttacksAndDetection with attack: JPEG__quality_40
Running VideoWatermarkAttacksAndDetection with attack: GaussianBlur__kernel_size_9
Running VideoWatermarkAttacksAndDetection with attack: MedianFilter__kernel_size_9
Running V

Result watermark_img_results exists and will be overriden as --overwrite is set
Result final_img_results exists and will be overriden as --overwrite is set
Running ImageWatermarkAttacksAndDetection with attack: no-attack
Running ImageWatermarkAttacksAndDetection with attack: proportion__prop_0.5
Running ImageWatermarkAttacksAndDetection with attack: collage__scale_1.0
Running ImageWatermarkAttacksAndDetection with attack: center_crop__scale_0.05
Running ImageWatermarkAttacksAndDetection with attack: jpeg__quality_factor_90
Running ImageWatermarkAttacksAndDetection with attack: sharpness__sharpness_factor_0.25
Running ImageWatermarkAttacksAndDetection with attack: resize__scale_0.89
Running ImageWatermarkAttacksAndDetection with attack: overlay_text
Running ImageWatermarkAttacksAndDetection with attack: hflip
Running ImageWatermarkAttacksAndDetection with attack: perspective__distortion_scale_0.4
Running ImageWatermarkAttacksAndDetection with attack: median_filter__kernel_size_3
Running

In [11]:
import os
def print_directory_tree(start_path, indent=""):
    for item in os.listdir(start_path):
        path = os.path.join(start_path, item)
        if os.path.isdir(path):
            print(f"{indent}📁 {item}/")
            print_directory_tree(path, indent + "    ")
        else:
            print(f"{indent}📄 {item}")

            
print_directory_tree("final_video_results")

📁 identity/
    📁 data/
        📄 wmd_video_00000.mp4
        📄 video_00000.mp4
    📁 detection/
        📄 detection_results_positive_0.pt
        📄 detection_results_negative_0.pt
    📄 attack_results.jsonl
    📄 metrics.json
📁 HorizontalFlip/
    📁 data/
        📄 wmd_video_00000.mp4
        📄 video_00000.mp4
    📁 detection/
        📄 detection_results_positive_0.pt
        📄 detection_results_negative_0.pt
    📄 attack_results.jsonl
    📄 metrics.json
📁 Resize__size_0.55/
    📁 data/
        📄 wmd_video_00000.mp4
        📄 video_00000.mp4
    📁 detection/
        📄 detection_results_positive_0.pt
        📄 detection_results_negative_0.pt
    📄 attack_results.jsonl
    📄 metrics.json
📁 Perspective__distortion_scale_0.5/
    📁 data/
        📄 wmd_video_00000.mp4
        📄 video_00000.mp4
    📁 detection/
        📄 detection_results_positive_0.pt
        📄 detection_results_negative_0.pt
    📄 attack_results.jsonl
    📄 metrics.json
📁 Brightness__factor_1.5/
    📁 data/
        📄 wmd_

### Explanation: 

- Instead of passing attacks name, we can pass a custom attack registry file to the `attacks` argument.

- The final results are stored in the directory specified by `result_dir`. This consists of several sub-directories, each correspond to a task. In the above example, the YAML file "video_attacks_mini.yaml" consists of 6 attacks, so we have 7 sub-directories (including the default "no-attack") under the result directory.

- For each task, we also have average metrics and raw results, similar to Example 1:
   - The average metrics are saved to the file "metrics.json". For each metric, the value is saved as a list of three values "avg", "count", "square", corresponding to the object `AverageMetric`. 
   - The raw results are saved to the file "attacks_results.jsonl". Since this is saved per attack, we have only one JSON line for each sub-directory.

- `cache_dir`: The watermark is only generated one time, and the output will be saved to `cache_dir` and are used by detection tasks of all attacks.

- `overwrite`: Whether we should overwrite all existing files and results in the `result_dir` and `cache_dir`. By default, if the task finds these existing results, it will just load them and skip the execution.

- `ids_to_save`: Choose which items to save. We can define to save all data to hard disk ("all"), or a list of specific ids ("0,1,2"), or a range of ids ("0-3") or mixed of them ("0,2,3,4-8").

### Cache:

> ⚠️ **Warning:** Right now the watermark generation WILL ALWAYS be cached before running the detection. If `cache_dir` is not specified, Omniseal Bench will use the default cache location in "~/.cache/omniseal/tmp" instead. The consequence is the directory size might grow significantly over time. 
We are implementing the "streaming" fashion ("keep_data_in_memory") to support quick analysis scenario on small dataset, where no data is cached. THis feature will be added soon, but until then, consider cleaning the cache after a while to save storage

### Final report:

The final report is saved to "report.csv" which consists of average metrics over all attack variants. This can also be obtained programmatically with "print_scores()" API:

In [5]:
report = e2e.print_scores(raw_results)
report

,watermark_det_score,watermark_det,fake_det_score,fake_det,bit_acc,word_acc,p_value,capacity,log10_p_value,psnr,ssim,decoder_time,qual_time,det_time,attack_time,idx,attack,attack_variant,cat
0,0.7500,True,0.375,False,0.8125,False,7.057334e-09,24.303017,-8.151359,46.470249,0.991911,0.0741,0.0952,0.0796,0.0000,0,identity,default,none
1,0.7500,True,0.375,False,0.8125,False,7.057334e-09,24.303017,-8.151359,46.470249,0.991911,0.0232,0.0544,0.0018,0.0000,1,identity,default,none
2,0.7500,True,0.375,False,0.8125,False,7.057334e-09,24.303017,-8.151359,46.470249,0.991911,0.0230,0.0538,0.0018,0.0000,2,identity,default,none
3,0.7500,True,0.375,False,0.8125,False,7.057334e-09,24.303017,-8.151359,46.470249,0.991911,0.0233,0.0557,0.0018,0.0000,3,identity,default,none
4,0.7500,True,0.375,False,0.8125,False,7.057334e-09,24.303017,-8.151359,46.470249,0.991911,0.0232,0.0563,0.0018,0.0000,4,identity,default,none
5,0.7500,True,0.375,False,0.8125,False,7.057334e-09,24.303017,-8.151359,46.470249,0.991911,0.0228,0.0551,0.0017,0.0000,5,identity,default,none
6,0.7500,True,0.375,False,0.8125,False,7.057334e-09,24.303017,-8.151359,46.470249,0.991911,0.0219,0.0527,0.0017,0.0000,6,identity,default,none
7,0.7500,True,0.375,False,0.8125,False,7.057334e-09,24.303017,-8.151359,46.470249,0.991911,0.0227,0.0527,0.0017,0.0000,7,identity,default,none
8,0.3125,False,0.375,False,0.6000,False,4.645594e-02,2.323952,-1.332959,40.695862,0.980614,0.0241,0.0593,0.0017,0.0735,0,JPEG,quality_40,Compression
9,0.3125,False,0.375,False,0.6000,False,4.645594e-02,2.323952,-1.332959,40.695862,0.980614,0.0232,0.0544,0.0018,0.0313,1,JPEG,quality_40,Compression


## Example 3: Generating watermarks only

If you want to generate watermarks only (and optionally look at the quality metrics) without running any attacks, use the "generation" task:

In [14]:
# In some CUDAs in FAIR cluster, calling AudioSeal compile() results in torch dynamo error, so we disable compile() with NO_TORCH_COMPILE
os.environ["NO_TORCH_COMPILE"] = "1"

generation_task = task(
    "generation",
    modality="video",
    # Dataset options
    dataset_dir="../examples/videos",
    
    # data loading options
    batch_size=2,
    num_workers=2,

    output_resolution=256,
    max_frames=5,
    fps=24,
    crf=0,    
    
    # watermark options:
    # message_size (Optional): THis is only specified if the model does not have the attribute 'nbits'
    how_to_generate_message="per_dataset",  # Options: 'per_dataset', 'per_batch'
    
    result_dir="watermark_video_results",
    overwrite=True,
    ids_to_save="all",  # Save all videos

    seed=42,
    metrics=["psnr", "ssim"],
)

generator = get_model(
    'videoseal_0.0',    
    attenuation="jnd_1_1",
    attenuation_config="../src/omnisealbench/models/videoseal_src/configs/attenuation.yaml",
    as_type="generator",
    device="cuda"
)

avg_metrics, raw_results = generation_task(generator)

omnisealbench.models.videoseal.build_model has the following dependencies: ['timm', 'pycocotools', 'einops', 'opencv-python', 'safetensors', 'av']
Result watermark_video_results exists and will be overriden as --overwrite is set


Result watermark_img_results exists and will be overriden as --overwrite is set



NOTE: We pass `as_type` argument to the `get_model()` call to indicate that we only load the generator checkpoint from the model. See [Model](Model.ipynb) for more details.

In [15]:
print_directory_tree("watermark_video_results")

📁 watermarks/
    📄 wmd_video_00000.mp4
    📄 message_0.txt
📄 watermark_results.jsonl
📄 metrics.json


The content of `avg_metrics` and `raw_results` are saved to two files `metrics.json` and `watermark_results.jsonl` accordingly.

For each of the saved items, we save the watermarked audio and secret message separately. Because we specify `how_to_generate_message="per_dataset"`, all "message_*.txt" should contain the same secret binary string.

If we want to save the original data too, specify the path in `data_subdir`:

In [16]:
generation_task_2 = task(
    "generation",
    modality="video",
    # Dataset options
    dataset_dir="../examples/videos",
    
    # data loading options
    batch_size=2,
    num_workers=2,

    output_resolution=256,
    max_frames=5,
    fps=24,
    crf=0,        
    
    # watermark options:
    # message_size (Optional): THis is only specified if the model does not have the attribute 'nbits'
    how_to_generate_message="per_dataset",  # Options: 'per_dataset', 'per_batch'
    
    result_dir="watermark_videos_results",
    data_subdir="data",
    overwrite=True,
    ids_to_save="all",  # Save all audios

    seed=42,
    metrics=["psnr", "ssim"],
)

generation_task_2(generator)

Result watermark_videos_results exists and will be overriden as --overwrite is set


({'psnr': AverageMetric(avg=87.6741943359375, count=1, square=7686.764352455735, avg_ci_fn=None),
  'ssim': AverageMetric(avg=0.9999973177909851, count=1, square=0.9999946355891645, avg_ci_fn=None),
  'qual_time': AverageMetric(avg=0.0075, count=1, square=5.625e-05, avg_ci_fn=None)},
 defaultdict(list,
             {'psnr': [87.6741943359375],
              'ssim': [0.9999973177909851],
              'qual_time': [0.0075],
              'idx': [0]}))

In [17]:
print_directory_tree("watermark_videos_results")

📁 watermarks/
    📄 wmd_video_00000.mp4
    📄 message_0.txt
📁 data/
    📄 video_00000.mp4
📄 watermark_results.jsonl
📄 metrics.json


Note the new sub-directory "data" with original image

## Example 4: Detection only
This scenario is useful when we just want to evaluate the model that generate the watermarks externally, for example the latent watermarking model.


In [ ]:
detection_task = task(
    "detection",
    modality="video",
    seed=42,
    dataset_dir="watermark_videos_results",
    original_video_pattern="data/*.mp4",
    watermarked_video_pattern="watermarks/*.mp4",  # We just fake the watermarked images
    message_pattern="watermarks/message_*.txt",
    metrics=["psnr"],  # We can add one more quality metric that was not previously computed
    result_dir="detection_video",
    overwrite=True,
    attacks=["JPEG", "GaussianBlur"],
    batch_size=2,
)

detector = get_model(
    'videoseal_0.0',    
    attenuation="jnd_1_1",
    attenuation_config="../src/omnisealbench/models/videoseal_src/configs/attenuation.yaml",
    as_type="detector",
    device="cuda"
)

avg_metrics, raw_results = detection_task(detector)

omnisealbench.models.videoseal.build_model has the following dependencies: ['timm', 'pycocotools', 'einops', 'opencv-python', 'safetensors', 'av']


Running ImageWatermarkAttacksAndDetection with attack: no-attack
Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_3
Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_5
Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_9
Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_13
Running ImageWatermarkAttacksAndDetection with attack: gaussian_blur__kernel_size_17
Skipping attack omnisealbench.attacks.image_effects.comb for ImageWatermarkAttacksAndDetection on DCTDWTDetector


In [ ]:
detection_task.print_scores(raw_results)